In [29]:
import opensimplex
import numpy as np
import matplotlib.pyplot as plt
import math
from random import randint
import os
import multiprocessing as mp
import plotly.express as px
from random import randint, random, uniform

In [30]:
ag = 32
biomas = np.empty(shape=(ag,ag,ag), dtype=np.uint8)
for i in range(ag):
    for j in range(ag):
        for k in range(ag):
            mejor = float('inf')
            for r in generator.BIOMAS.keys():
                a = np.linalg.norm(np.array(generator.BIOMAS[r][0]) - np.array([i/ag, j/ag, k/ag]))
                if a < mejor:
                    mejor = a
                    biomas[i, j, k] = r
    print(i)
print(biomas)
np.save(f"CERCANIA_BIOMAS.npy", biomas)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
[[[10 10 10 ... 11 11 11]
  [10 10 10 ... 11 11 11]
  [10 10 10 ... 11 11 11]
  ...
  [14 14 14 ... 16 16 16]
  [14 14 14 ... 16 16 16]
  [14 14 14 ... 16 16 16]]

 [[10 10 10 ... 11 11 11]
  [10 10 10 ... 11 11 11]
  [10 10 10 ... 11 11 11]
  ...
  [14 14 14 ... 16 16 16]
  [14 14 14 ... 16 16 16]
  [14 14 14 ... 16 16 16]]

 [[10 10 10 ... 11 11 11]
  [10 10 10 ... 11 11 11]
  [10 10 10 ... 11 11 11]
  ...
  [14 14 14 ... 16 16 16]
  [14 14 14 ... 16 16 16]
  [14 14 14 ... 16 16 16]]

 ...

 [[ 5  5  5 ... 12 12 12]
  [ 5  5  5 ... 12 12 12]
  [ 5  5  5 ... 12 12 12]
  ...
  [15 15 15 ... 12 12 12]
  [15 15 15 ... 12 12 12]
  [15 15 15 ... 12 12 12]]

 [[ 5  5  5 ... 12 12 12]
  [ 5  5  5 ... 12 12 12]
  [ 5  5  5 ... 12 12 12]
  ...
  [15 15 15 ... 12 12 12]
  [15 15 15 ... 12 12 12]
  [15 15 15 ... 12 12 12]]

 [[ 5  5  5 ... 12 12 12]
  [ 5  5  5 ... 12 12 12]
  [ 5  5  5 ... 12 12 12]
  ...
  [1

In [31]:
TEXTURAS_SUELO = {
    0: "Water",
    1: "Grass",
    2: "Sand",
    3: "Stone",
    4: "Brown Stone",
    5: "Volcanic Stone",
    6: "Snow",
}

TEXTURAS = {
    "Arbol": {
        0: "Palmera", #Desierto
        1: "Roble", #Valle
        2: "Sauce", #Pantano
        3: "Arbol de selva", #Selva
        4: "Acacia", #Savana
        5: "Encina", #Dehesa
        6: "Conífero", #Bosque
        6: "Árbol carbonizado", #Volcanico
    },
    "Roca": { #Para cada bioma habrá un tipo
        "Roca pequeña": {

        },
        "Roca mediana": {

        },
        "Roca grande": {

        },
    }
}

In [32]:
class generator:
    """
    A class to generate terrain using OpenSimplex noise and other procedural generation techniques.
    Attributes:
    -----------
    BIOMAS : dict
        A dictionary containing biome information with keys as biome IDs and values as tuples containing
        humidity, altitude, temperature, biome name, color, and objects with their probabilities.
    CHUNK_SIZE : int
        The size of each chunk in the terrain.
    CERCANIA_BIOMAS : np.ndarray
        A numpy array loaded from a file that contains biome proximity data.
    Methods:
    --------
    __init__(seedTemp: int = None, seedAltu: int = None, seedHume: int = None, seedRios: int = None, 
             varTemp: int = 256, varAltu: int = 512, varHume: int = 512, varRios: int = 128, 
             dispTemp: int = 2, dispAltu: int = 4, dispHume: int = 2, dispRios: int = 4, 
             nivelAgua: int = 0.5, tamRios: int = 5):
        Initializes the generator with given seeds, variations, disparities, water level, and river size.
    getNoise(seed: int, x_in: int, y_in: int, iterations: int, size: int, disparity: int = 3) -> np.float16:
        Generates Perlin noise for a given seed, coordinates, iterations, size, and disparity.
    getNoiseArray(seed: int, x_in: int, y_in: int, iterations: int, size: int, disparity: int = 3) -> np.ndarray:
        Generates a 2D array of Perlin noise for a given seed, coordinates, iterations, size, and disparity.
    getBioma(temp: float, altu: float, hume: float, rios: float) -> np.uint8:
        Determines the biome type based on temperature, altitude, humidity, and river presence.
    getChunk(x: int, y: int) -> np.ndarray:
        Retrieves or generates a chunk of terrain data for given coordinates.
    getChunksInRange(x_range: tuple[int, int], y_range: tuple[int, int]) -> np.ndarray:
        Retrieves or generates multiple chunks of terrain data within a specified range of coordinates.
    representation(x_range: tuple[int, int], y_range: tuple[int, int]) -> None:
        Generates a visual representation of the terrain within a specified range of coordinates.
    poisson_disc_sampling(radius: int, k: int) -> np.ndarray:
        Generates a 2D array using Poisson-disc sampling for a given radius and number of attempts.
    """
    #(Humedad altura temperatura), nombre del bioma, color, {objetos que pueden aparecer, probabilidades}
    BIOMAS = {
        0: ([-1, -1, -1], 'Oceano', (56, 148, 194), {}),
        1: ([-1, -1, -1], 'Oceano profundo', (27, 59, 140), {}),
        2: ([-1, -1, -1], 'Rios y lagos', (126, 180, 237), {}),
        3: ([0.5, 0.0, 0.5], 'Costa', (245, 240, 108), {"Palmera": 0.7, "Piedra pequeña": 0.3}),
        4: ([0.85, 0.05, 0.45], 'Marismas', (171, 222, 32), {"Arbusto": 1}),
        5: ([0.75, 0.1, 0.05], 'Polo', (250, 250, 250), {"Tempano": 0.5, "Piedra congelada": 0.5}),
        6: ([0.8, 0.15, 0.45], 'Pantano', (33, 133, 99), {"Arbusto": 0.2, "Piedra pequeña": 0.2, "Arbol": 0.6}),
        7: ([0.5, 0.2, 0.5], 'Bosque', (53, 118, 43), {"Arbol": 0.6, "Arbusto": 0.2, "Flores": 0.2}),
        8: ([0.6, 0.2, 0.55], 'Selva', (0, 183, 10), {"Arbol": 0.7, "Helecho": 0.3}),
        9: ([0.35, 0.2, 0.55], 'Dehesa', (128, 179, 30), {"Arbol": 0.5, "Arbusto": 0.5}),
        10: ([0.2, 0.2, 0.6], 'Savana', (228, 125, 28), {"Arbol": 0.3, "Arbusto": 0.7}),
        12: ([0.75, 0.25, 0.75], 'Termales', (39, 156, 121), {"Geiser": 0.5, "Piedra pequeña": 0.5}),
        11: ([0.0, 0.3, 0.8], 'Desierto', (255, 211, 29), {"Cactus": 0.7, "Arbusto muerto": 0.3}),
        13: ([0.8, 0.3, 0.3], 'Setas', (152, 3, 252), {"Seta grande": 0.6, "Seta pequeña": 0.4}),
        14: ([0.2, 0.65, 0.55], 'Meseta', (110, 88, 66), {"Roca grande": 0.1, "Arbusto": 0.5, "Roca pequeña": 0.4}),
        15: ([0.7, 0.8, 0.2], 'Tundra', (208, 208, 240), {"Piedra pequeña": 0.5, "Flores": 0.5}),
        16: ([0.15, 0.8, 0.85], 'Volcanico', (159, 16, 16), {"Arbol carbonizado": 0.6, "Charco de lava": 0.4}),
        17: ([0.35, 9, 0.35], 'Montaña', (125, 125, 125), {"Roca grande": 0.3, "Roca pequeña": 0.7}),
    }

    CHUNK_SIZE = 128
    CERCANIA_BIOMAS = np.load("./CERCANIA_BIOMAS.npy" )

    def __init__(
        self, 
        seedTemp: int = None, seedAltu: int = None, seedHume: int = None, seedRios: int = None,
        varTemp: int = 256, varAltu: int = 512, varHume: int = 512, varRios: int = 128,
        dispTemp: int = 2, dispAltu: int = 4, dispHume: int = 2, dispRios: int = 4,
        nivelAgua: int = 0.5,
        tamRios: int = 5,
        itTemp: int = 3, itAltu: int = 5, itHume: int = 3, itRios: int = 4,
        ):
        """
        Initialize the PerlinNoise generator with various parameters.
        Args:
            seedTemp (int, optional): Seed for temperature noise generation. Defaults to None.
            seedAltu (int, optional): Seed for altitude noise generation. Defaults to None.
            seedHume (int, optional): Seed for humidity noise generation. Defaults to None.
            seedRios (int, optional): Seed for river noise generation. Defaults to None.
            varTemp (int, optional): Variation for temperature noise. Defaults to 256.
            varAltu (int, optional): Variation for altitude noise. Defaults to 512.
            varHume (int, optional): Variation for humidity noise. Defaults to 512.
            varRios (int, optional): Variation for river noise. Defaults to 128.
            dispTemp (int, optional): Displacement for temperature noise. Defaults to 2.
            dispAltu (int, optional): Displacement for altitude noise. Defaults to 4.
            dispHume (int, optional): Displacement for humidity noise. Defaults to 2.
            dispRios (int, optional): Displacement for river noise. Defaults to 4.
            nivelAgua (float, optional): Water level threshold. Defaults to 0.5.
            tamRios (int, optional): Size of rivers. Defaults to 5.
            itTemp (int, optional): Iterations for temperature noise generation. Defaults to 3.
            itAltu (int, optional): Iterations for altitude noise generation. Defaults to 6.
            itHume (int, optional): Iterations for humidity noise generation. Defaults to 3.
            itRios (int, optional): Iterations for river noise generation. Defaults to 4.
        """
        self.SEEDTEMP = seedTemp if seedTemp != None else randint(-2**62, 2**62)
        self.SEEDALTU = seedAltu if seedTemp != None else randint(-2**62, 2**62)
        self.SEEDHUME = seedHume if seedTemp != None else randint(-2**62, 2**62)
        self.SEEDRIOS = seedRios if seedTemp != None else randint(-2**62, 2**62)
        self.VARTEMP = varTemp
        self.VARALTU = varAltu
        self.VARHUME = varHume
        self.VARRIOS = varRios
        self.DISPTEMP = dispTemp
        self.DISPALTU = dispAltu
        self.DISPHUME = dispHume
        self.DISPRIOS = dispRios
        self.NIVELAGUA = nivelAgua
        self.TAMRIOS = tamRios
        self.ITTEMP = itTemp
        self.ITALTU = itAltu
        self.ITHUME = itHume
        self.ITRIOS = itRios

    def getNoise(self, seed: int, x_in: int, y_in: int, iterations: int, size: int, disparity: int = 3) -> np.float16:
        """
        Generates Perlin noise based on the given parameters.
        Args:
            seed (int): The seed value for the noise generation.
            x_in (int): The x-coordinate input for the noise function.
            y_in (int): The y-coordinate input for the noise function.
            iterations (int): The number of iterations to perform for noise generation.
            size (int): The size parameter that affects the scale of the noise.
            disparity (int, optional): The number of disparity iterations to smooth the noise. Default is 3.
        Returns:
            np.float16: The generated noise value.
        """
        opensimplex.seed(seed)
        ruido = 0
        for r in range(iterations):
            divisor = (size/2**r)
            exponenciador = (2**r)
            ruido += opensimplex.noise2(x=(x_in)/divisor, y=(y_in)/divisor)/exponenciador
            
        maximo = sum([2/(2**r) for r in range(iterations)])
        ruido = (ruido + maximo)/(2*maximo)     
        for r in range(disparity):
            ruido = 0.5 + math.sin(math.pi*ruido - math.pi/2)/2
        return ruido

    def getNoiseArray(
        self, 
        seed: np.int64, 
        x_in: np.int64, 
        y_in: np.int64, 
        iterations: np.uint8, 
        size: np.uint8, 
        disparity: np.uint8 = 3
        ) -> np.ndarray:
        """
        Generates a 2D array of Perlin noise values.
        Args:
            seed (np.int64): Seed for the noise generation.
            x_in (np.int64): X-coordinate input for the chunk.
            y_in (np.int64): Y-coordinate input for the chunk.
            iterations (np.uint8): Number of iterations for noise generation.
            size (np.uint8): Size parameter for noise scaling.
            disparity (np.uint8, optional): Number of disparity adjustments to apply. Default is 3.
        Returns:
            np.ndarray: A 2D array of generated noise values.
        """
        x = self.CHUNK_SIZE*x_in
        y = self.CHUNK_SIZE*y_in
        opensimplex.seed(seed)

        rango = range(0,self.CHUNK_SIZE)
        noise_array = np.zeros(dtype=np.float16, shape=(self.CHUNK_SIZE, self.CHUNK_SIZE))

        for r in range(iterations):
            divisor = (size/2**r)
            exponenciador = (2**r)
            for i in rango:
                for j in rango:
                    noise_array[j, i] += opensimplex.noise2(x=(i+x)/divisor, y=(y+j)/divisor)/exponenciador
        
        maximo = sum([2/(2**r) for r in range(iterations)])
        for i in rango:
            for j in rango:
                valor = (noise_array[j, i] + maximo)/(2*maximo)     
                for r in range(disparity):
                    valor = 0.5 + math.sin(math.pi*valor - math.pi/2)/2
                noise_array[j, i] = valor
        return noise_array
    
    def getBioma(
        self, 
        temp: np.float16, 
        altu: np.float16, 
        hume: np.float16, 
        rios: np.float16
        ) -> np.uint8:
        """
        Determines the biome type based on temperature, altitude, humidity, and proximity to rivers.
        Args:
            temp (np.float16): Temperature value.
            altu (np.float16): Altitude value.
            hume (np.float16): Humidity value.
            rios (np.float16): Proximity to rivers value.
        Returns:
            np.uint8: Biome type identifier.
            - 0: Ocean
            - 1: Deep Ocean
            - 2: Rivers and Lakes
            - Other values based on the CERCANIA_BIOMAS matrix.
        """
        if(altu < self.NIVELAGUA):
            if (altu <= self.NIVELAGUA*0.8):
                return 1 #Oceano profundo
            else:
                return 0 #Oceano
        elif(0.5 - self.TAMRIOS/100 < rios < 0.5 + self.TAMRIOS/100):
            return 2 #Rios y lagos
        else:
            alt = math.floor(self.CERCANIA_BIOMAS.shape[0]*(altu-self.NIVELAGUA)/self.NIVELAGUA)
            temp = math.floor(self.CERCANIA_BIOMAS.shape[1]*temp)
            hum = math.floor(self.CERCANIA_BIOMAS.shape[2]*hume)
            return np.uint8(self.CERCANIA_BIOMAS[hum, alt, temp])

    def getChunk(self, x: np.int16, y: np.int16) -> np.ndarray:
        """
        Generates or loads a chunk of terrain data based on the given coordinates.

        This method checks if the chunk data for the specified coordinates (x, y) already exists.
        If it does, the data is loaded from a .npy file. If not, the method generates the chunk
        data using Perlin noise and saves it to a .npy file for future use.

        Args:
            x (np.int16): The x-coordinate of the chunk.
            y (bp.int16): The y-coordinate of the chunk.

        Returns:
            np.ndarray: A 2D array representing the generated or loaded chunk data.
        """
        if os.path.exists(f"./Chunks/T_{self.SEEDTEMP}A_{self.SEEDTEMP}H_{self.SEEDTEMP}/{x}/{y}.npy"):
            return np.load(f"./Chunks/T_{self.SEEDTEMP}A_{self.SEEDTEMP}H_{self.SEEDTEMP}/{x}/{y}.npy")
        else:
            temp = self.getNoiseArray(self.SEEDTEMP, x, y, self.ITTEMP, self.VARTEMP, self.DISPTEMP)
            altu = self.getNoiseArray(self.SEEDALTU, x, y, self.ITALTU, self.VARALTU, self.DISPALTU)
            hume = self.getNoiseArray(self.SEEDHUME, x, y, self.ITHUME, self.VARHUME, self.DISPHUME)
            rios = self.getNoiseArray(self.SEEDRIOS, x, y, self.ITRIOS, self.VARRIOS, self.DISPRIOS)
            array_biomas = np.zeros(dtype=np.uint8, shape=(self.CHUNK_SIZE,self.CHUNK_SIZE))

            for i in range(self.CHUNK_SIZE):
                for j in range(self.CHUNK_SIZE):
                    array_biomas[i, j] = self.getBioma(hume[i, j], altu[i, j], temp[i, j], rios[i, j])
            os.makedirs(f"./Chunks/T_{self.SEEDTEMP}A_{self.SEEDTEMP}H_{self.SEEDTEMP}/{x}", exist_ok=True)
            np.save(f"./Chunks/T_{self.SEEDTEMP}A_{self.SEEDTEMP}H_{self.SEEDTEMP}/{x}/{y}", array_biomas)
            return array_biomas
    
    def getChunksInRange(self, x_range: tuple[np.int16, np.int16], y_range: tuple[np.int16, np.int16]) -> np.ndarray:
        arr = None
        for i in range(x_range[0], x_range[1]):
            arr_proc = []
            for j in range(y_range[0], y_range[1]):
                arr_proc.append(mp.Process(target=self.getChunk, args=(i, j)))
                arr_proc[-1].start()
            for proc in arr_proc:
                proc.join()

            arr_line = None
            for j in range(y_range[0], y_range[1]):
                if arr_line is None:
                    arr_line = self.getChunk(i, j)
                else:
                    arr_line = np.vstack((arr_line, self.getChunk(i, j)))

            if arr is None:
                arr = arr_line
            else:
                arr = np.hstack((arr, arr_line))
        return arr

    def representation(self, x_range: tuple[int, int], y_range: tuple[int, int]) -> None:
        arr = self.getChunksInRange(x_range, y_range)
        color_arr = np.zeros((arr.shape[0], arr.shape[1], 3), dtype=np.uint8)

        for i in range(arr.shape[0]):
            for j in range(arr.shape[1]):
                color_arr[i, j] = generator.BIOMAS[arr[i, j]][2]
        
        fig = px.imshow(color_arr)
        # Adjust the figure size here, considering the legend size
        fig.update_layout(
            width=arr.shape[1] + 300,  # Add extra width for the legend
            height=arr.shape[0] + 300  # Add extra height for the legend
        )

        # Add legend
        legend_items = []
        for key, value in generator.BIOMAS.items():
            legend_items.append(
                dict(
                    name=value[1],
                    marker=dict(color=f"rgb{value[2]}", size=20),
                    mode='markers',
                    type='scatter',
                    x=[None],
                    y=[None]
                )
            )
        fig.add_traces(legend_items)
        fig.show()
    
    def poisson_disc_sampling(self, radius: int, k: int) -> np.ndarray:
        GRID_SIZE = int(radius / math.sqrt(2))
        columnas, filas = self.CHUNK_SIZE // GRID_SIZE, self.CHUNK_SIZE // GRID_SIZE

        def generate_point_around(point):
            r = radius * (random() + 1)
            angle = 2 * math.pi * random()
            new_x = point[0] + r * math.cos(angle)
            new_y = point[1] + r * math.sin(angle)
            return new_x, new_y

        def in_bounds(point):
            return 0 <= point[0] < self.CHUNK_SIZE and 0 <= point[1] < self.CHUNK_SIZE

        def fits(point):
            col = int(point[0] / GRID_SIZE)
            row = int(point[1] / GRID_SIZE)
            for i in range(max(col - 2, 0), min(col + 3, columnas)):
                for j in range(max(row - 2, 0), min(row + 3, filas)):
                    neighbor = grid[i, j]
                    if neighbor is not None and np.hypot(point[0] - neighbor[0], point[1] - neighbor[1]) < radius:
                        return False
            return True

        def restart_simulation(start_point):
            nonlocal grid, active, points
            grid = np.empty((columnas, filas), dtype=object)
            points = [start_point]
            active = [start_point]
            col = int(start_point[0] / GRID_SIZE)
            row = int(start_point[1] / GRID_SIZE)
            grid[col, row] = start_point

        # Create a grid to store points
        grid = np.zeros((columnas, filas), dtype=object)

        # List to store active points
        active = []
        points = []

        # Initialize with a random point
        initial_point = (uniform(0, self.CHUNK_SIZE), uniform(0, self.CHUNK_SIZE))
        restart_simulation(initial_point)

        # Main loop
        while active:
            rand_index = randint(0, len(active) - 1)
            point = active[rand_index]
            found = False

            for _ in range(k):
                new_point = generate_point_around(point)
                if in_bounds(new_point) and fits(new_point):
                    points.append(new_point)
                    active.append(new_point)
                    col = int(new_point[0] / GRID_SIZE)
                    row = int(new_point[1] / GRID_SIZE)
                    grid[col, row] = new_point
                    found = True
                    break

            if not found:
                active.pop(rand_index)
                # Create a 2D numpy array representation
        matrix = np.zeros((self.CHUNK_SIZE, self.CHUNK_SIZE), dtype=np.uint8)
        for p in points:
            x, y = int(p[0]), int(p[1])
            matrix[y, x] = 1

        return matrix


In [33]:
a = generator(
    seedTemp=1353,
    seedAltu=13312,
    seedHume=3123,
    seedRios=3134,
    tamRios=7,
    varRios=128
    )

In [34]:
a.representation((0,8),(0,8))

In [35]:
resultado = a.poisson_disc_sampling(3,10)
for r in resultado:
    print("".join([str("🌲" if j==1 else "🟫") for j in list(r.flatten())][0:64]))

🟫🟫🟫🟫🌲🟫🟫🟫🟫🟫🌲🟫🟫🟫🌲🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🌲🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🌲🟫🟫🟫🌲
🌲🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🌲🟫🟫🌲🟫🟫🟫🟫🟫🌲🟫🟫🌲🟫🟫🟫🌲🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫
🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🌲🟫🟫🌲🟫🟫🟫🌲🟫🟫🟫🟫🌲🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🌲🟫🟫
🟫🟫🟫🟫🟫🟫🟫🌲🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🌲🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🌲🟫🟫🟫🟫🟫🟫🟫
🟫🟫🟫🌲🟫🟫🟫🟫🟫🟫🌲🟫🟫🟫🟫🟫🌲🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🌲🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫
🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🌲🟫🟫🟫🟫🟫🟫🟫🌲🟫🟫🟫🌲🟫🟫🌲🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🌲🟫🟫🟫🟫🟫🟫🟫🌲🟫🟫🟫🟫
🌲🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🌲🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🌲🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🌲🟫
🟫🟫🟫🟫🟫🟫🟫🌲🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🌲🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🌲🟫🟫🌲🟫🟫🟫🟫🟫🟫
🟫🟫🟫🟫🌲🟫🟫🟫🟫🟫🟫🟫🟫🌲🟫🟫🟫🟫🌲🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🌲🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🌲🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫
🌲🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🌲🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🌲🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫
🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🌲🟫🟫🟫🟫🟫🟫🌲🟫🟫🌲🟫🟫🟫🟫🟫🟫🟫🟫🌲🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🌲🟫🟫🌲🟫
🟫🟫🟫🟫🟫🟫🌲🟫🟫🟫🌲🟫🟫🟫🟫🟫🟫🟫🟫🟫🌲🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🌲🟫🟫🌲🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫
🟫🌲🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🌲🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🌲🟫🟫🟫🟫🟫🟫🟫🟫🌲🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🌲🟫🟫🟫🟫🟫🟫🟫🟫
🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🌲🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🌲🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫
🟫🟫🟫🟫🟫🟫🌲🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🌲🟫🟫🟫🟫🟫🌲🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫
🟫🟫🟫🟫🟫🟫🟫🟫🟫🌲🟫🟫🌲🟫🟫🟫🟫🌲🟫🟫🟫🟫🟫🟫🟫